In [1]:
import os
import pandas as pd

In [13]:
def load_landmarks(txt_file):
    with open(txt_file, 'r') as f:
        lines = f.readlines()
    landmarks = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) >= 2:
            x = float(parts[0])
            y = float(parts[1])
            landmarks.append((x, y))
    return landmarks if len(landmarks) == 49 else None


In [14]:
def extract_age_from_filename(filename):
    try:
        base = os.path.splitext(filename)[0]
        age = int(base.split("A")[-1])  # 예: "02665A27" → "27"
        return age
    except:
        return None


In [17]:
def create_aaf_csv(aligned_dir, keypoint_dir, save_path):
    data = []

    for filename in sorted(os.listdir(aligned_dir)):
        if not filename.lower().endswith(".jpg"):
            continue

        base_name = os.path.splitext(filename)[0]
        img_path = os.path.join(aligned_dir, filename)
        kp_path = os.path.join(keypoint_dir, base_name + ".txt")

        if not os.path.exists(kp_path):
            continue

        age = extract_age_from_filename(filename)
        if age is None:
            continue

        landmarks = load_landmarks(kp_path)
        if landmarks is None:
            continue

        row = {"image_path": img_path, "age": age}
        for i, (x, y) in enumerate(landmarks):
            row[f"x{i}"] = x
            row[f"y{i}"] = y

        data.append(row)

    df = pd.DataFrame(data)
    df.to_csv(save_path, index=False)
    print(f"✅ CSV saved to '{save_path}' with {len(df)} samples.")


In [26]:
import os
import pandas as pd

# 설정
image_dir = "/Users/geko/COSE471_data-science/AFF/aglined_faces"
output_csv = "aaf_age_labels.csv"

data = []

# 파일 이름에서 나이 추출
for filename in sorted(os.listdir(image_dir)):
    if not filename.lower().endswith(".jpg"):
        continue
    try:
        age_str = filename.split("A")[-1].replace(".jpg", "")
        age = int(age_str)
        path = os.path.join(image_dir, filename)
        data.append({"image_path": path, "age": age})
    except:
        continue  # 예외가 있으면 스킵

# 저장
df = pd.DataFrame(data)
df.to_csv(output_csv, index=False)
print(f"✅ Saved {len(df)} samples to {output_csv}")

✅ Saved 13322 samples to aaf_age_labels.csv


In [18]:
create_aaf_csv(
    aligned_dir="/Users/geko/COSE471_data-science/AFF/aglined_faces",  # 오타라면 실제 폴더명에 맞게 수정!
    keypoint_dir="/Users/geko/COSE471_data-science/AFF/key_points",
    save_path="aaf_age_landmarks.csv"
)


✅ CSV saved to 'aaf_age_landmarks.csv' with 13322 samples.
